# Pipelines workshop 2: creating an ABT

In the first notebook for this workshop, "importing and preparing the data" you read in data from an external source and processed it into two CSV files: data/processed/income.csv and data/processed/expenditure.csv.

Now we are going to combine these two CSV files into one large table so that we can analyze the data. Such a combined table is known as an Analytical Base Table (ABT).

As this notebook, too, is part of our pipeline, we will be rigorously logging  all the unexpected situations we encounter in our work.




In [2]:
import pandas as pd
from lib.utils import *
from loguru import logger

## Read in our data

Reading in our data should not cause problems, but just in case it does: log errors and halt execution if one of our two data files does not exist.

- Load income data into a dataframe `dfi`
- Load expenditure data into a dataframe `dfe`

In [2]:
# Your code goes here

In [3]:
try:
    dfi = pd.read_csv("data/processed/income.csv")
except Exception as e:
    logger.critical(f"Unable to read income csv file: {e}")
    exit(0)

try:
    dfe = pd.read_csv("data/processed/expenditure.csv")
except Exception as e:
    logger.critical(f"Unable to read expenditure csv file: {e}")
    exit(0)